In [ ]:
# default_exp core

# nbdev_try2.core

> Give it another try.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
say_hello("Sara")

'Hello Sara!'

Let's try how to use a database in a way that it will not break CI tests on GitHub...

In [ ]:
import sqlite3
import os

In [ ]:
DBNAME = "example.sqlite"
if os.path.exists(DBNAME):
    print("removing", DBNAME)
    os.remove(DBNAME)
    
conn = sqlite3.connect(DBNAME)
cur = conn.cursor()
cur.execute("""
    create table if not exists numbers (
        number integer primary key,
        name
    )
""")
cur.execute("""
    insert into numbers (number, name) 
        values (1, "One"),
            (2, "Two"),
            (3, "Three"),
            (17, "Seventeen")
""")
resu = cur.execute("""
    select name from numbers where number = ?
""", (17, )).fetchall()

assert len(resu) == 1, len(resu)
assert resu[0][0] == "Seventeen"

print(resu)

removing example.sqlite
[('Seventeen',)]


This is working. Also repetitive, because it removes tha database before. So let's try to push it...

Tests succeeded in GitHub, but we have no clue whether it was executed :(

So let's try with something failing...

In [ ]:
assert 1 == 0, "deliberately failing"

AssertionError: deliberately failing

This actually passes `nbdev_test_nbs` successfully.

Docs say:

> Everything that is not an exported cell is considered a test

So, again, something plain wrong :(